In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import numpy as np
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.linalg as la
import pyvista as pv
import imageio.v3 as iio
np.set_printoptions(formatter={"float": "{:10.2f}".format})

In [63]:
from src.pyushichka import loadCalibration

K_my, P_my = loadCalibration(0, r"C:\data\ushichka\2018-08-17")
print(f"P\n{P_my}")

D: 3.289521239260813, Uo: 320.04939830822735, Vo:257.9544556169163
[[     -0.60      -0.79       0.14      -0.08]
 [     -0.23       0.34       0.91      -1.01]
 [      0.77      -0.52       0.38       3.29]
 [      0.00       0.00       0.00       1.00]]
P
[[    -69.85    -579.58     195.67    1008.88]
 [     77.68      44.47     578.10     309.54]
 [      0.77      -0.52       0.38       3.29]]


In [34]:
extr = np.vstack((la.inv(K_my)@P_my, [0,0,0,1]))
print(extr)
pose = la.inv(extr)
pose

[[     -0.18      -0.24       0.04      -0.03]
 [     -0.07       0.10       0.28      -0.30]
 [      0.23      -0.16       0.12       1.00]
 [      0.00       0.00       0.00       1.00]]


array([[     -1.97,      -0.74,       2.53,      -2.80],
       [     -2.59,       1.11,      -1.70,       1.97],
       [      0.46,       3.01,       1.24,      -0.31],
       [      0.00,       0.00,       0.00,       1.00]])

In [61]:
def cam_centre_from_dlt(coefs):
    '''
    
    
    Reference
    ---------
    * http://www.kwon3d.com/theory/dlt/dlt.html Equation 25
    '''
        
    m1 = np.array([[coefs[0],coefs[1],coefs[2]],
                 [coefs[4],coefs[5],coefs[6]],
                 [coefs[8],coefs[9],coefs[10]]])
    m2 = np.array([-coefs[3], -coefs[7], -1]).T

    xyz = np.matmul(np.linalg.inv(m1),m2)
    return xyz

def transformation_matrix_from_dlt(coefs):
    '''
    Based on the DLTcameraPosition.m function written by Ty Hedrick. 
    
    Parameters
    ----------
    coefs : (11,Ncamera) np.array
    
    Returns
    -------
    T : (4,4) np.array
        Transformation matrix
    Z : float
        Distance of camera centre behind image plane
    ypr : (3,) np.array
        yaw, pitch, roll angles in degrees
    
    
    Notes
    -----
    I guess this function is based on the equations described in 
    Kwon3d (http://www.kwon3d.com/theory/dlt/dlt.html#3d).
            
    The transformation matrix T -
    
    
    
    ''' 
    D = (1/(coefs[8]**2+coefs[9]**2+coefs[10]**2))**0.5;
    #D = D[0]; # + solution
    
    Uo=(D**2)*(coefs[0]*coefs[8]+coefs[1]*coefs[9]+coefs[2]*coefs[10]);
    Vo=(D**2)*(coefs[4]*coefs[8]+coefs[5]*coefs[9]+coefs[6]*coefs[10]);
    print(f'D: {D}, Uo: {Uo}, Vo:{Vo}')
    du = (((Uo*coefs[8]-coefs[0])**2 + (Uo*coefs[9]-coefs[1])**2 + (Uo*coefs[10]-coefs[2])**2)*D**2)**0.5;
    dv = (((Vo*coefs[8]-coefs[4])**2 + (Vo*coefs[9]-coefs[5])**2 + (Vo*coefs[10]-coefs[6])**2)*D**2)**0.5;
    
    #du = du[0]; # + values
    #dv = dv[0]; 
    Z = -1*np.mean([du,dv]) # there should be only a tiny difference between du & dv
    
    row1 = [(Uo*coefs[8]-coefs[0])/du ,(Uo*coefs[9]-coefs[1])/du ,(Uo*coefs[10]-coefs[2])/du]
    row2 = [(Vo*coefs[8]-coefs[4])/dv ,(Vo*coefs[9]-coefs[5])/dv ,(Vo*coefs[10]-coefs[6])/dv] 
    row3 = [coefs[8] , coefs[9], coefs[10]]
    T3 = D*np.array([row1,
                     row2,
                     row3])

    dT3 = np.linalg.det(T3);
    
    if dT3 < 0:
      T3=-1*T3;
    
    xyz = cam_centre_from_dlt(coefs)
    
    T = np.zeros((4,4))
    T[:3,:3] = np.linalg.inv(T3);
    T[3,:]= [xyz[0], xyz[1], xyz[2], 1]
    
        
    # % compute YPR from T3
    # %
    # % Note that the axes of the DLT based transformation matrix are rarely
    # % orthogonal, so these angles are only an approximation of the correct
    # % transformation matrix
    # %  - Addendum: the nonlinear constraint used in mdlt_computeCoefficients below ensures the
    # %  orthogonality of the transformation matrix axes, so no problem here
    alpha = np.arctan2(T[1,0],T[0,0])
    beta = np.arctan2(-T[2,0], (T[2,1]**2+T[2,2]**2)**0.5)
    gamma = np.arctan2(T[2,1],T[2,2])
    
    ypr = np.rad2deg([gamma,beta,alpha]);

    return T, Z, ypr

def make_rotation_mat_fromworld(Rc, C):
    '''
    Parameters
    ----------
    Rc : 3x3 np.array
        Rotation matrix wrt the world coordinate system
    C : (1,3) or (3,) np.array
        Camera XYZ in world coordinate system
    Returns
    -------
    camera_rotation: 4x4 np.array
        The final camera rotation and translation matrix which 
        converts the world point to the camera point
    
    References
    ----------
    * Simek, Kyle, https://ksimek.github.io/2012/08/22/extrinsic/ (blog post) accessed 14/2/2022
    
    Example
    -------
    
    > import track2trajectory.synthetic_data as syndata
    > Rc, C = ..... # load and define the world system camera rotations and camera centre
    > rotation_mat = make_rotation_mat_fromworld(Rc, C)
    '''
    camera_rotation = np.zeros((4,4))
    camera_rotation[:3,:3] = Rc.T
    camera_rotation[:3,-1] = -np.matmul(Rc.T,C)
    camera_rotation[-1,-1] = 1 
    return camera_rotation

#print(P_my.flatten()[:-1])
camera_pose_T , _, _ = transformation_matrix_from_dlt(P_my.flatten()[:-1])
shifter_mat = np.row_stack(([-1,0,0,0],
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]))

#print(camera_pose_T)

shifted_rotmat = np.matmul(camera_pose_T, shifter_mat)[:3,:3]
extrinsic_matrix = make_rotation_mat_fromworld(shifted_rotmat, camera_pose_T[-1,:3])

print(extrinsic_matrix)
P = K_my @ extrinsic_matrix[:3,:]
P


D: 3.289521239260813, Uo: 320.04939830822735, Vo:257.9544556169163
[[     -0.60      -0.79       0.14      -0.08]
 [     -0.23       0.34       0.91      -1.01]
 [      0.77      -0.52       0.38       3.29]
 [      0.00       0.00       0.00       1.00]]


array([[    -69.85,    -579.58,     195.67,    1008.88],
       [     77.68,      44.47,     578.10,     309.54],
       [      0.77,      -0.52,       0.38,       3.29]])

In [64]:
la.det((la.inv(K_my) @P)[:3,:3])

-0.9999999841675165